In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/Project/'
#%pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: '/MyDrive/Project/'
/content


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [ ]:
np.random.seed(682)
tf.random.set_seed(682)

In [ ]:
class Distiller(keras.Model):
  
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [ ]:
teacher = tf.keras.models.load_model('/saved_models/resnet50cifar')

In [ ]:
def preprocess_image_input(input_images):
  input_images = tf.cast(input_images, 'float32')
  output_ims = tf.keras.applications.mobilenet.preprocess_input(input_images)
  return output_ims

class Preprocess(tf.keras.layers.Layer):
    def __init__(self):
        super(Preprocess, self).__init__()

    def call(self, inputs):
        return preprocess_image_input(inputs)

student_mobile = tf.keras.applications.MobileNet(
    input_shape=(224, 224, 3),
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=True,
    weights=None,
    input_tensor=None,
    pooling=None,
    classes=10,
    classifier_activation=None
)

inputs = tf.keras.layers.Input(shape=(32,32,3))
resize = tf.keras.layers.UpSampling2D(size=(7,7))(inputs)
pre_process = Preprocess()(resize)
resnet_extractor = student_mobile(pre_process)
student = tf.keras.Model(inputs=inputs, outputs = resnet_extractor)

In [ ]:
def preprocess_image_input(input_images):
  input_images = tf.cast(input_images, 'float32')
  output_ims = tf.keras.applications.mobilenet.preprocess_input(input_images)
  return output_ims

class Preprocess(tf.keras.layers.Layer):
    def __init__(self):
        super(Preprocess, self).__init__()

    def call(self, inputs):
        return preprocess_image_input(inputs)

student_mobile = tf.keras.applications.MobileNet(
    input_shape=(224, 224, 3),
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=True,
    weights=None,
    input_tensor=None,
    pooling=None,
    classes=10,
    classifier_activation=None
)

inputs = tf.keras.layers.Input(shape=(32,32,3))
resize = tf.keras.layers.UpSampling2D(size=(7,7))(inputs)
pre_process = Preprocess()(resize)
resnet_extractor = student_mobile(pre_process)
student_scratch = tf.keras.Model(inputs=inputs, outputs = resnet_extractor)

In [ ]:
# # Create the student
# student = vgg16_model()

# # Clone student for later comparison
# student_scratch = keras.models.clone_model(student)

In [ ]:
# Prepare the train and test dataset.
batch_size = 64

(x_train, y_train) , (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student, teacher=teacher)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Distill teacher to student
distiller.fit(x_train, y_train, epochs=5)

# Evaluate student on test dataset
distiller.evaluate(x_test, y_test)

Epoch 1/5
1563/1563 [==============================] - 303s 191ms/step - sparse_categorical_accuracy: 0.5095 - student_loss: 1.3751 - distillation_loss: 0.0091
Epoch 2/5
1563/1563 [==============================] - 297s 190ms/step - sparse_categorical_accuracy: 0.6924 - student_loss: 0.9133 - distillation_loss: 0.0116
Epoch 3/5
1563/1563 [==============================] - 297s 190ms/step - sparse_categorical_accuracy: 0.7727 - student_loss: 0.7054 - distillation_loss: 0.0127
Epoch 4/5
1563/1563 [==============================] - 297s 190ms/step - sparse_categorical_accuracy: 0.8225 - student_loss: 0.5665 - distillation_loss: 0.0131
Epoch 5/5
313/313 [==============================] - 8s 24ms/step - sparse_categorical_accuracy: 0.8075 - student_loss: 0.5868


[0.8075000047683716, 0.4203187823295593]

In [ ]:
student.save('/content/drive/MyDrive/Project/models/saved_models/ts_3')

student_test = tf.keras.models.load_model('/content/drive/MyDrive/Project/models/saved_models/ts_3')

# Initialize and compile distiller
distiller_test = Distiller(student=student_test, teacher=teacher)
metric_student = keras.metrics.SparseCategoricalAccuracy(name='s1_acc')

distiller_test.compile (
    optimizer=keras.optimizers.Adam(),
    metrics=[metric_student],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Evaluate student on test dataset
distiller_test.evaluate(x_test, y_test)

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Project/models/saved_models/ts_3/assets
313/313 [==============================] - 8s 25ms/step - s1_acc: 0.8075 - student_loss: 0.5868


[0.8075000047683716, 0.4203187823295593]

In [ ]:
# Train student as doen usually
student_scratch.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate student trained from scratch.
student_scratch.fit(x_train, y_train, epochs=5)
student_scratch.evaluate(x_test, y_test)

Epoch 1/5
1563/1563 [==============================] - 226s 142ms/step - loss: 1.3687 - sparse_categorical_accuracy: 0.5075
Epoch 2/5
1563/1563 [==============================] - 223s 143ms/step - loss: 0.9185 - sparse_categorical_accuracy: 0.6768
Epoch 3/5
1563/1563 [==============================] - 223s 143ms/step - loss: 0.7015 - sparse_categorical_accuracy: 0.7548
Epoch 4/5
1563/1563 [==============================] - 223s 143ms/step - loss: 0.5528 - sparse_categorical_accuracy: 0.8079
Epoch 5/5
313/313 [==============================] - 8s 23ms/step - loss: 0.6637 - sparse_categorical_accuracy: 0.7816


[0.6636810302734375, 0.7815999984741211]

In [ ]:
student_scratch.save('/content/drive/MyDrive/Project/models/saved_models/scratch_3')

student_scratch_test = tf.keras.models.load_model('/content/drive/MyDrive/Project/models/saved_models/scratch_3')

# Train student as doen usually
student_scratch_test.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate student trained from scratch.
student_scratch_test.evaluate(x_test, y_test)

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Project/models/saved_models/scratch_3/assets
313/313 [==============================] - 8s 23ms/step - loss: 0.6637 - sparse_categorical_accuracy: 0.7816


[0.6636810302734375, 0.7815999984741211]